In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import os 
import logging

In [2]:
class SerieASpider(scrapy.Spider):
    name = "Serie A"
    start_urls = ["https://www.lequipe.fr/Football/championnat-d-italie/page-calendrier-resultats"]

    def parse(self, response):
       card = response.xpath('//*[@id="outer-edito-content"]/div[2]/main/div[1]/div/label/select/option')
       for i in card:
            yield {
                #"url": i.xpath('option').attrib["value"]
                "url":i.xpath('./@value').get()
            }

# Define a filename
filename = "serie_A_scrap_url.json"

# Remove previous file if it allready exist
if filename in os.listdir('src/'):
        os.remove('src/' + filename)

# Define the settings for the crawler process
process = CrawlerProcess(settings = {
    # simulate browser
    'USER_AGENT': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0', 
    # display regular errors
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'src/' + filename : {"format": "json"},
    }
})

# Initiate crawler
process.crawl(SerieASpider)
process.start()

2022-11-08 10:06:36 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2022-11-08 10:06:36 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.4.0, Python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Windows-10-10.0.19044-SP0
2022-11-08 10:06:36 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) '
               'Gecko/20100101 Firefox/98.0'}
2022-11-08 10:06:36 [scrapy.extensions.telnet] INFO: Telnet Password: a1aa1a0724f02908
2022-11-08 10:06:36 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-11-08 10:06:36 [scrapy.middleware] INFO: Enabled downloader middleware

In [11]:
import pandas as pd
import json

In [15]:
file = open('src/serie_A_scrap_url.json')

In [16]:
serieA_url = json.load(file)

In [29]:
serieA_url[0]

{'url': 'https://iphdata.lequipe.fr/iPhoneDatas/EFR/STD/ALL/V1/Football/CalendarList/CompetitionPhase/championnat-d-italie/current/1re-journee.json'}

In [31]:
serieA_url[0]['url']

'https://iphdata.lequipe.fr/iPhoneDatas/EFR/STD/ALL/V1/Football/CalendarList/CompetitionPhase/championnat-d-italie/current/1re-journee.json'

In [33]:
URLS = []
for i in range(13):
    url = serieA_url[i]['url']
    URLS.append(url)

In [ ]:
class SerieASpidermatch(scrapy.Spider):
    name = "Serie A"
    URLS = serieA_url

    def start_requests(self):
        for d in [URLS]:
            yield Request(d,callback=self.parse)

    def parse(self, response):
        card = response.xpath('//*[@id="outer-edito-content"]/div[3]/main/div[3]/div[2]/div[1]/div/div')
        for i in card:
            winner = i.css('div.TeamScore__name TeamScore__team--winner').get()
            looser = i.css('TeamScore__name TeamScore__team--looser').get()
            score = i.css('TeamScore__score TeamScore__score--ended').getall()
            dic = {
                'winner': winner , 'looser': looser , 'score': score
            }
            
            
            yield dic
    
    

# Define a filename
filename = "serie_A_scrap_matches.json"

# Remove previous file if it allready exist
if filename in os.listdir('src/'):
        os.remove('src/' + filename)

# Define the settings for the crawler process
process = CrawlerProcess(settings = {
    # simulate browser
    'USER_AGENT': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0', 
    # display regular errors
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'src/' + filename : {"format": "json"},
    }
})

# Initiate crawler
process.crawl(SerieASpidermatch)
process.start()